#### We start by importing the relevant libraries

In [1]:
!jt -l

Available Themes: 
   chesterish
   grade3
   gruvboxd
   gruvboxl
   monokai
   oceans16
   onedork
   solarizedd
   solarizedl


#### We start by importing the relevant libraries

In [3]:
import scapy.all as scapy
import numpy as np 
import graphviz

## We load the pcap file and split it into Packets and Frames:

'0:1'

In [68]:
all_traffic = scapy.rdpcap("./pcap/network-traffic.pcap")
packet_list = []		# to store layer 3 traffic
filt = []
for pkt in all_traffic:
    if pkt.haslayer('IP'):
        filt.append([pkt['IP']])
filtered = (pkt for pkt in all_traffic if IP in pkt)
for packet in all_traffic:
    if (scapy.IP in packet):
        packet_list.append([packet['IP']])
packet_list = np.array(packet_list)

/tmp/ipykernel_10612/4180136061.py:11: VisibleDeprecationWarning: Creating an ndarray from nested sequences exceeding the maximum number of dimensions of 32 is deprecated. If you mean to do this, you must specify 'dtype=object' when creating the ndarray.
  packet_list = np.array(packet_list)


In [70]:
print(packet_list[0])
print(len(packet_list))
print(filt[0])
print(len(filt))

[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[<IP  version=4 ihl=5 tos=0x0 len=61 id=38507 flags=DF frag=0 ttl=64 proto=udp chksum=0x6750 src=10.138.38.223 dst=10.139.1.1 |<UDP  sport=53822 dport=domain len=41 chksum=0x3d2f |<DNS  id=63604 qr=0 opcode=QUERY aa=0 tc=0 rd=1 ra=0 z=0 ad=0 cd=0 rcode=ok qdcount=1 ancount=0 nscount=0 arcount=0 qd=<DNSQR  qname='www.tor2web.org.' qtype=A qclass=IN |> an=None ns=None ar=None |>>>]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]
658
[<IP  version=4 ihl=5 tos=0x0 len=61 id=38507 flags=DF frag=0 ttl=64 proto=udp chksum=0x6750 src=10.138.38.223 dst=10.139.1.1 |<UDP  sport=53822 dport=domain len=41 chksum=0x3d2f |<DNS  id=63604 qr=0 opcode=QUERY aa=0 tc=0 rd=1 ra=0 z=0 ad=0 cd=0 rcode=ok qdcount=1 ancount=0 nscount=0 arcount=0 qd=<DNSQR  qname='www.tor2web.org.' qtype=A qclass=IN |> an=None ns=None ar=None |>>>]
658


In [36]:
engines = ["dot", "twopi", "neato", "circo"]
for item in engines:
    print(item)

dot
twopi
neato
circo


## Identify the conversation (communication between end points)

In [5]:
conv_packets = list(set(packet_list[:,0]))
conv_frames = list(set(frame_list[:,0]))

In [6]:
print(packet_list[0])

[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[<Ether  dst=fe:ff:ff:ff:ff:ff src=00:16:3e:5e:6c:00 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=61 id=38507 flags=DF frag=0 ttl=64 proto=udp chksum=0x6750 src=10.138.38.223 dst=10.139.1.1 |<UDP  sport=53822 dport=domain len=41 chksum=0x3d2f |<DNS  id=63604 qr=0 opcode=QUERY aa=0 tc=0 rd=1 ra=0 z=0 ad=0 cd=0 rcode=ok qdcount=1 ancount=0 nscount=0 arcount=0 qd=<DNSQR  qname='www.tor2web.org.' qtype=A qclass=IN |> an=None ns=None ar=None |>>>>]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]


In [6]:
matrix_layer3 = np.transpose(np.array([conv_packets, [0] * len(conv_packets)]))
matrix_layer2 = np.transpose(np.array([conv_frames, [0] * len(conv_frames)]))

### Calculation of the conversation size

In [7]:
for i in range(len(conv_frames)):
	conversation_size = 0
	for j in range(len(frame_list[:,0])):
		if (matrix_layer2[i,0] == frame_list[j,0]):
			conversation_size += int(frame_list[j,1])
	matrix_layer2[i,1] = conversation_size

In [8]:
for i in range(len(conv_packets)):
	conversation_size = 0
	for j in range(len(packet_list[:,0])):
		if (matrix_layer3[i,0] == packet_list[j,0]):
			conversation_size += int(packet_list[j,1])
	matrix_layer3[i,1] = conversation_size

## We create three vectors for Source, Destination and ConversationSize

In [9]:
src_macs = []
dst_macs = []
con_macs = []
for i in range (len(matrix_layer2)):
	macs = matrix_layer2[i,0].split(",")
	src_macs.append(macs[0])
	dst_macs.append(macs[1])
	con_macs.append(matrix_layer2[i,1])

In [10]:
con_macs

['126', '70', '126', '70']

In [11]:
src_ips = []
dst_ips = []
con_siz = []
for i in range (len(matrix_layer3)):
	ips = matrix_layer3[i,0].split(",")
	src_ips.append(ips[0])
	dst_ips.append(ips[1])
	con_siz.append(matrix_layer3[i,1])

## We create a dictionary of the conversation with prefix PC<number>

In [12]:
mac_values = list(set(src_macs + dst_macs))
ip_values = list(set(src_ips + dst_ips))

In [13]:
pc_labels2 = []
for i in range (1, len(mac_values) + 1):
	prefix = "MAC" + str(i)
	pc_labels2.append(prefix)
mac_dict = dict(zip(mac_values, pc_labels2))

In [14]:
pc_labels = []
for i in range (1, len(ip_values) + 1):
	prefix = "IP" + str(i)
	pc_labels.append(prefix)
conversation_dict = dict(zip(ip_values, pc_labels))

## We substitute IP values for their respective key

In [15]:
# for item in src_macs:
# 	index = src_macs.index(item)
# 	src_macs[index] = mac_dict.get(item)
# for item in dst_macs:
# 	index = dst_macs.index(item)
# 	dst_macs[index] = mac_dict.get(item)

In [16]:
# for item in src_ips:
# 	index = src_ips.index(item)
# 	src_ips[index] = conversation_dict.get(item)
# for item in dst_ips:
# 	index = dst_ips.index(item)
# 	dst_ips[index] = conversation_dict.get(item)

## We create the Digraph of IPv4 Conversations and Ethernet Conversations

In [17]:
dotether = graphviz.Digraph(comment='Ethernet Conversations')
dotether.engine = "twopi" #doctest: +ELLIPSIS
dotether.graph_attr['ranksep']='2'		#separation between center (twopi engine)
dotether.graph_attr['nodesep']='0.5'		#separation between arrows

In [18]:
for key, value in mac_dict.items():
	dotether.node(key, value)

In [23]:
con_macs

['126', '70', '126', '70']

In [19]:
con_siz_int2 = list(map(int, con_macs))
A2 = min(con_siz_int2)
B2 = max(con_siz_int2)
P2 = 1  # thinest edge
Q2 = 12  # thickest edge
b2 = (Q2 - P2) / (B2 - A2)
a2 = P2 - b2 * A2
mapped_con_size2 = [a2 + b2 * x for x in con_siz_int2]
str_map_con_size2 = list(map(str, mapped_con_size2))

In [22]:
str_map_con_size2

['12.0', '1.0', '12.0', '1.0']

In [20]:
for i in range(len(src_macs)):
	dotether.edge(src_macs[i], dst_macs[i], penwidth=str_map_con_size2[i])

In [21]:
dotether.format = 'svg'
dotether.render('test-output/conversation-ether.gv')

Error: conversation-ether.gv: syntax error in line 7 near ':'


CalledProcessError: Command '[WindowsPath('dot'), '-Ktwopi', '-Tsvg', '-O', 'conversation-ether.gv']' returned non-zero exit status 1. [stderr: b"Warning: syntax ambiguity - badly delimited number '3e' in line 7 of conversation-ether.gv splits into two tokens\r\nWarning: syntax ambiguity - badly delimited number '5e' in line 7 of conversation-ether.gv splits into two tokens\r\nWarning: syntax ambiguity - badly delimited number '6c' in line 7 of conversation-ether.gv splits into two tokens\r\nError: conversation-ether.gv: syntax error in line 7 near ':'\r\n"]

In [ ]:
'test-output/conversation-ether.gv.svg'

In [ ]:
# dotipv4 = graphviz.Digraph(comment='IPv4 Conversations')
# dotipv4.engine = "twopi" #doctest: +ELLIPSIS
# dotipv4 #doctest: +ELLIPSIS
# dotipv4.graph_attr['ranksep']='2'		#separation between center (twopi engine)
# dotipv4.graph_attr['nodesep']='0.5'		#separation between arrows

In [ ]:
# for key, value in conversation_dict.items():
# 	dotipv4.node(key, value)

In [ ]:
# con_siz_int = list(map(int, con_siz))
# A = min(con_siz_int)
# B = max(con_siz_int)
# P = 1  # thinest edge
# Q = 12  # thickest edge
# b = (Q - P) / (B - A)
# a = P - b * A
# mapped_con_size = [a + b * x for x in con_siz_int]
# mapped_con_size= list(map(str, mapped_con_size))

In [ ]:
# for i in range(len(src_ips)):
# 	dotipv4.edge(src_ips[i], dst_ips[i], penwidth=str_map_con_size[i])

In [ ]:
# dotipv4.format = 'svg'
# dotipv4.render('test-output/conversation-ipv4.gv')

In [ ]:
# 'test-output/conversation-ipv4.gv.svg'

In [ ]:
# readable_size = []
# for item in con_siz_int:
# 	units_size = 1
# 	counter = 0
# 	while (item / (1000 * units_size) > 1):
# 		units_size = units_size * 1000
# 		counter += 1
# 	output = item / units_size
# 	if (counter == 0):
# 		prefix = "B"
# 	elif (counter == 1):
# 		prefix = "KB"
# 	elif (counter == 2):
# 		prefix = "MB"
# 	elif (counter == 3):
# 		prefix = "GB"
# 	elif (counter == 4):
# 		prefix = "TB"
# 	if (counter == 0):
# 		read = str(int(output)) + " " + prefix
# 	else:
# 		read = "{:.2f}".format(output) + " " + prefix
# 	readable_size.append(read)

## We make an create a dictionary of ocurrences (relevant for analysis of frequency)

In [ ]:
# key_occurrences = []
# value_ocurrences = []
# for key, value in conversation_dict.items():
# 	counter = 0
# 	key_occurrences.append(value)
# 	for item in src_ips_copy:
# 		counter = counter + 1 if (key == item) else counter
# 	value_ocurrences.append(counter)
# occurrences_dict = dict(zip(key_occurrences, value_ocurrences))